# A Scrap-reducing Scheduling Strategy for Batch Production

A spreadsheet containing one year of operational data indicates several reasons for scrap. In this notebook, we assume that the following values listed in the column **scrap reasons** indicate pigment from the prior batch contaminating the next batch, creating the scrap:
* Tile Shade
* Tile Quality
* Offshade
* Foreign Color
* Patt. Change

These scrap reasons account for 15.6% of total scrap. 

We model the scheduling problem as a [Traveling Salesman Problem](https://en.wikipedia.org/wiki/Travelling_salesman_problem). Where replacing "distance between cities" is one of:

1. an empirical model of "scrap between product types" or, 
2. an analytical model describing the difference between color as it affects scrap, or
3. some hybrid combination of (1) and (2). 

To use the empirical approach, ideally we would have accurate historical operational data indicating the scrap produced between every pair of products. Of course, we don't have that. 

## Do it in MiniZinc!

First we have to load some "MiniZinc Magic" that allows the notebook to know that code cells starting with `%%minizinc` are to be processes by the MiniZinc solver. 

In [1]:
 %load_ext iminizinc

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.2.1, build 29527117
Copyright (C) 2014-2018 Monash University, NICTA, Data61


I got this code from [these slides from Bologna Business School](http://cs.unibo.it/~tong.liu3/mzn/slides_mzn.pdf). If this doesn't do what we want, let's check out [this solution on GitHub](https://github.com/hakank/hakank/blob/master/minizinc/tsp.mzn).

In [9]:
import pandas as pd
from geopy.distance import great_circle

In [54]:
cities = pd.read_csv("data/city.csv")

In [55]:
cities

,City,Latitude,Longitude
0,Montgomery,32.361538,-86.279118
1,Phoenix,33.448457,-112.073844
2,Little Rock,34.736009,-92.331122
3,Sacramento,38.555605,-121.468926
4,Denver,39.739167,-104.984167
5,Hartford,41.767000,-72.677000
6,Dover,39.161921,-75.526755
7,Tallahassee,30.451800,-84.272770
8,Atlanta,33.760000,-84.390000
9,Boise,43.613739,-116.237651


In [61]:
cdict = cities.set_index('City').T.to_dict('list')

In [58]:
cities.columns

Index(['City', 'Latitude', 'Longitude'], dtype='object')

In [62]:
cdict["Hartford"]

[41.766999999999996, -72.67699999999999]

In [53]:
tuple([1,2])

(1, 2)

In [66]:
from geopy.distance import great_circle
print(great_circle(tuple(cdict["Hartford"]),tuple(cdict["Springfield"])))
print(great_circle(tuple(cdict["Hartford"]),tuple(cdict["Springfield"])).miles)

1443.7343180504388 km
897.0949144809554


In [ ]:
%%minizinc -m bind

include "globals.mzn";

int n;
array [1..n,1..n] of int: dist;
int: start_city;
int end_city;

array[1..n] of var 1..n city;
array[1..n] of string: city_name;
    
constraint city[1] = start_city;
constraint city[n] = end_city;
constraint all_different(city);

int total_distance = sum(i in 2..n)(dist[city[i-1],city[i]]);
solve minimize total_distance;

## Never Stop Improving!

![alt text](https://raw.githubusercontent.com/pdenno/minizinc-notebooks/master/images/flexecution.jpg "Logo Title Text 1")

We can do "flexecution" inside the notebook by deciding what goals are important, elaborating a plan and pursuing it. 

Some NIST software will help. 

## Bibliography

* [GitHub site for TSP solved for 48 US States.](https://github.com/toddwschneider/shiny-salesman) But the data is in binary. 
* [Here is longitudes and latitudes](http://www.xfront.com/us_states/), then use [this](https://pypi.org/project/geopy/) to calculate the distances. 